In [1]:
%%capture

import warnings

warnings.filterwarnings("ignore")

import _section1_utils as section1
import _section2_utils as section2
import calitp_data_analysis.magics
import great_tables as gt
import pandas as pd
import yaml
from IPython.display import HTML, Image, Markdown, display, display_html
from shared_utils import catalog_utils, rt_dates, rt_utils

with open("readable.yml") as f:
    readable_dict = yaml.safe_load(f)

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
# Comment out and leave this cell right below pandas
organization_name = "Yolo County Transportation District"

In [4]:
# %%capture_parameters
# organization_name

In [5]:
most_recent_date = pd.to_datetime(rt_dates.DATES["mar2024"])
year = most_recent_date.year

In [6]:
# Testing certain operators
# organization_name = "City and County of San Francisco"

In [7]:
# Load first dataset
operator_profiles = section1.load_operator_profiles(organization_name)

In [8]:
# Need to load "name" using this function rather than taking it from
# operator profiles above because it differs.
name = section1.organization_name_crosswalk(organization_name)

In [9]:
GTFS_DATA_DICT = catalog_utils.get_catalog("gtfs_analytics_data")

In [10]:
# Datasets for Section 1
operator_route_map = section1.load_operator_map(name)
monthly_scheduled_service = section1.summarize_monthly(name)

In [11]:
# Datasets for Section 2
sched_vp_df = section2.load_schedule_vp_metrics(organization_name)
gtfs_schedule_keys = list(sched_vp_df.schedule_gtfs_dataset_key.unique())

In [12]:
# Set drop down menu to be on the upper right
display(
    HTML(
        """
<style>
form.vega-bindings {
  position: absolute;
  right: 0px;
  top: 0px;
}
</style>
"""
    )
)

# {organization_name}

In [13]:
display(
    Markdown(
        f"""<h2 style="border-bottom: 3px solid #fdcb79;"><b>Operator Overview</b></h2>
        """
    )
)

<h2 style="border-bottom: 3px solid #fdcb79;"><b>Operator Overview</b></h2>
        

In [14]:
try:
    date = operator_profiles["Date"].dt.to_pydatetime()[0]
except:
    pass

In [15]:
try:
    display(
        Markdown(
            f"""This section presents an overview of statistics for {organization_name}, using data from the most recent date of
      {date.strftime("%B")}, {date.year}."""
        )
    )
except:
    pass

This section presents an overview of statistics for Yolo County Transportation District, using data from the most recent date of
      April, 2024.

In [16]:
try:
    n_routes = operator_profiles["# Routes"].values[0]
    n_routes = "{:,}".format(n_routes)
    display(
        Markdown(
            f"""{organization_name} ran <b>{n_routes}</b> 
        unique routes. Below is the breakdown of the routes."""
        )
    )
except:
    display(Markdown(f"""{organization_name} doesn't have an operator profile."""))

Yolo County Transportation District ran <b>19</b> 
        unique routes. Below is the breakdown of the routes.

In [17]:
try:
    route_typology_df = section1.route_typology(operator_profiles)
    display(
        section1.basic_pie_chart(
            route_typology_df,
            "route_type",
            "total_routes",
            readable_dict["route_typology"]["title"],
            readable_dict["route_typology"]["subtitle"],
        )
    )

except:
    display(
        Markdown(f"""{organization_name} doesn't have route typologies available.""")
    )

alt.Chart(...)

In [18]:
try:
    n_service_miles = int(operator_profiles["Service Miles"].values[0])
    n_service_miles = "{:,}".format(n_service_miles)

    avg_service_miles = int(operator_profiles["Service Miles"].mean())
    avg_service_miles = "{:,}".format(avg_service_miles)
    display(
        Markdown(
            f"""{organization_name} provided <b>{n_service_miles}</b> miles of public transit.
        The average length of a route is <b>{avg_service_miles}</b> miles.
        """
        )
    )
except:
    pass

In [19]:
try:
    longest_shortest_df = section1.shortest_longest_route(operator_route_map)
    display(
        section1.basic_bar_chart(
            longest_shortest_df,
            "Service Miles",
            "Route",
            readable_dict["longest_shortest_route"]["title"],
            readable_dict["longest_shortest_route"]["subtitle"],
        )
    )
except:
    pass

alt.Chart(...)

In [20]:
display(
    Markdown(f"{organization_name} provides public transit in the following counties:")
)

Yolo County Transportation District provides public transit in the following counties:

In [21]:
try:
    section1.counties_served(operator_route_map)
except:
    display(Markdown(f"""{organization_name} doesn't have an route geographies."""))

-Yolo
-Sacramento
-Solano


In [22]:
display(Markdown(f"Map of all the routes."))
try:
    display(
        operator_route_map.explore(
            "Route",
            cmap="Spectral",
            tiles="CartoDB positron",
            width=500,
            height=300,
            style_kwds={"weight": 3},
            legend=False,
            tooltip=["Route", "Service Miles"],
        )
    )
    


except:
    section1.interactive_map(operator_route_map)

Map of all the routes.

In [23]:
try:
    n_stops = int(operator_profiles["# Stops"].values[0])
    n_stops = "{:,}".format(n_stops)

    n_arrivals = int(operator_profiles["# Arrivals"].values[0])
    n_arrivals = "{:,}".format(n_arrivals)

    mean_arrivals = int(operator_profiles["Avg Arrivals per Stop"].values[0])
    mean_arrivals = "{:,}".format(mean_arrivals)
    display(
        Markdown(
            f"""{organization_name} stopped at <b>{n_stops}</b> unique stops, 
            totaling to <b>{n_arrivals}</b> arrivals. The average arrivals per stop 
            was <b>{mean_arrivals}</b>.
        """
        )
    )
except:
    pass

Yolo County Transportation District stopped at <b>386</b> unique stops, 
            totaling to <b>6,427</b> arrivals. The average arrivals per stop 
            was <b>16</b>.
        

In [24]:
display(
    Markdown(
        f"Use the dropdown menu on the right to select a date to view total scheduled service hours by month."
    )
)

Use the dropdown menu on the right to select a date to view total scheduled service hours by month.

In [25]:
monthly_scheduled_service_df_agg = section1.summarize_monthly(name)

In [26]:
section1.single_bar_chart_dropdown(
    monthly_scheduled_service_df_agg,
    "Day of Week",
    "Total Service (hours)",
    "Time of Day",
    readable_dict["total_scheduled_hours"]["title"],
    "full_date",
    readable_dict["total_scheduled_hours"]["subtitle"],
)

alt.Chart(...)

In [27]:
display(
    Markdown(
        f"""<h2 style="border-bottom: 3px solid #fdcb79;"><b>Detailed Route Overview</b></h2><br>
               
        """
    )
)

<h2 style="border-bottom: 3px solid #fdcb79;"><b>Detailed Route Overview</b></h2><br>
               
        

In [28]:
section2.trips_by_gtfs(sched_vp_df)

GT(_tbl_data=GTFS Availability       Date  schedule_only  vp_only  schedule_and_vp
0                 2023-05-17              0        0              172
1                 2023-06-14              0        0              171
2                 2023-07-12              0        0              171
3                 2023-08-15              0        0              172
4                 2023-09-13              0        0              172
5                 2023-10-11              0        0              168
6                 2023-11-15              0        0              172
7                 2023-12-13              0        0              170
8                 2024-01-17              0        0              172
9                 2024-02-14              0        0              178
10                2024-03-13              0        0              178
11                2024-04-17              0        0              178, _body=<great_tables._gt_data.Body object at 0x7f7d2eb23220>, _boxhead=Boxhead([ColInfo(var='Date', type=<ColInfoTypeEnum.stub: 2>, column_label='Date', column_align='right', column_width=None), ColInfo(var='schedule_only', type=<ColInfoTypeEnum.default: 1>, column_label='Schedule Only', column_align='right', column_width=None), ColInfo(var='vp_only', type=<ColInfoTypeEnum.default: 1>, column_label='VP Only', column_align='right', column_width=None), ColInfo(var='schedule_and_vp', type=<ColInfoTypeEnum.default: 1>, column_label='Schedule and VP', column_align='right', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=Timestamp('2023-05-17 00:00:00'), group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=Timestamp('2023-06-14 00:00:00'), group_label=None, built=False), RowInfo(rownum_i=2, group_id=None, rowname=Timestamp('2023-07-12 00:00:00'), group_label=None, built=False), RowInfo(rownum_i=3, group_id=None, rowname=Timestamp('2023-08-15 00:00:00'), group_label=None, built=False), RowInfo(rownum_i=4, group_id=None, rowname=Timestamp('2023-09-13 00:00:00'), group_label=None, built=False), RowInfo(rownum_i=5, group_id=None, rowname=Timestamp('2023-10-11 00:00:00'), group_label=None, built=False), RowInfo(rownum_i=6, group_id=None, rowname=Timestamp('2023-11-15 00:00:00'), group_label=None, built=False), RowInfo(rownum_i=7, group_id=None, rowname=Timestamp('2023-12-13 00:00:00'), group_label=None, built=False), RowInfo(rownum_i=8, group_id=None, rowname=Timestamp('2024-01-17 00:00:00'), group_label=None, built=False), RowInfo(rownum_i=9, group_id=None, rowname=Timestamp('2024-02-14 00:00:00'), group_label=None, built=False), RowInfo(rownum_i=10, group_id=None, rowname=Timestamp('2024-03-13 00:00:00'), group_label=None, built=False), RowInfo(rownum_i=11, group_id=None, rowname=Timestamp('2024-04-17 00:00:00'), group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([]), _heading=Heading(title='Daily Trips by GTFS Availability', subtitle='Schedule only indicates the trip(s) were found only in schedule data. Vehicle Positions (VP) only indicates the trip(s) were found only in real-time data.', preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x7f7cb7410340>, _formats=[<great_tables._gt_data.FormatInfo object at 0x7f7cb7410b50>], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False,

In [29]:
display(
    Markdown(
        f"""Statistics presented on the <b>route</b> level. Use the dropdown menu on the right to select a specific route. 
           """
    )
)

Statistics presented on the <b>route</b> level. Use the dropdown menu on the right to select a specific route. 
           

In [30]:
section2.filtered_route_test(sched_vp_df)

alt.VConcatChart(...)